<a href="https://colab.research.google.com/github/FranMoretti/FranMoretti/blob/main/00_evaluaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Servidor

In [ ]:
from flask import Flask, jsonify, abort

In [ ]:
from flask import Flask, jsonify, abort, request, render_template
from os import path
import sqlite3

app = Flask(__name__)

THIS_FOLDER = path.dirname(path.realpath(__file__))

@app.route('/api/version', methods=["GET"])
def api_version():
    return jsonify({"version": 2})

@app.route('/api/init', methods=["GET"])
def api_init():
    con = sqlite3.connect(path.join(THIS_FOLDER, "data.db"))
    con.row_factory = sqlite3.Row
    cur = con.cursor()

    cur.execute("DROP TABLE IF EXISTS popularidad")
    con.commit()
    cur.execute("""CREATE TABLE popularidad AS
    SELECT id_libro, AVG(rating) AS rating
      FROM interacciones
     GROUP BY 1
    HAVING count(*) > 5
    """)
    cur.execute("CREATE UNIQUE INDEX idx_id_libro ON popularidad(id_libro)")
    con.close()
    return jsonify({"status": "ok"})


def recomendar(id_lector):
    con = sqlite3.connect(path.join(THIS_FOLDER, "data.db"))
    con.row_factory = sqlite3.Row
    cur = con.cursor()

    res = cur.execute("SELECT * FROM lectores WHERE id_lector = ?", [id_lector]).fetchone()
    if not res:
        abort(404, "El lector '{id_lector}' no existe")

    q = """
         SELECT id_libro
           FROM popularidad
           WHERE id_libro NOT IN (SELECT id_libro FROM interacciones WHERE id_lector = ?)
         ORDER BY rating DESC
         LIMIT 10;
    """
    cur.execute(q, [id_lector])
    rec = [r['id_libro'] for r in cur.fetchall()]
    con.close()

    return rec

@app.route("/api/recomendar/<string:id_lector>", methods=["GET"])
def api_recomendar(id_lector):
    return jsonify({"recomendacion": recomendar(id_lector)})

@app.route("/api/recomendaciones", methods=["POST"])
def api_recomendaciones():
    data = request.get_json()
    recs = []

    for id_lector in data["id_lectores"]:
        recs.append({"id_lector": id_lector, "recomendaciones": recomendar(id_lector)})

    return jsonify({"recomendaciones": recs})

## Cliente

In [ ]:
import requests
import sqlite3
import math
from os import path

In [ ]:
def ndcg(groud_truth, recommendation):
    dcg = 0.0
    for i, r in enumerate(recommendation):
        rel = int(r in groud_truth) # 0 si r no está en GT, 1 si está
        dcg += rel / math.log2(i+1+1)

    rels = [r for r in recommendation if r in groud_truth]
    not_rels = [r for r in recommendation if r not in groud_truth]

    idcg = 0.0
    for i, r in enumerate(rels + not_rels):
        idcg += 1.0 / math.log2(i+1+1)

    return dcg / idcg

In [ ]:
servidores = [
    (">>>>>>>>", "rabalde.pythonanywhere.com"),
    ("Amado, Pablo Marcelo", "pamado.pythonanywhere.com"),
    ("Bessone, Cristian", "v817723.pythonanywhere.com"),
    ("Brazionis, Camila Belén", "camilabrazionis.pythonanywhere.com"),
    ("Buccari, María Florencia", "florbuccari.pythonanywhere.com"),
    ("Cabrera, Tomas", "tomascabrera.pythonanywhere.com"),
    ("Caldora, Facundo", "facaldora.pythonanywhere.com/"),
    ("Chaparro, Raul Martín", "chapa.pythonanywhere.com"),
    ("Choi, Francisco", "fchoi.pythonanywhere.com"),
    ("Colina, Ariel Nicolás", "acolina.pythonanywhere.com"),
    ("Correa, Tomás", "tomascorrea.pythonanywhere.com"),
    ("Delgadin, Tomas Horacio", "tomasdelgadin.pythonanywhere.com"),
    ("Fernández Palacios, Tomás", "tofernan.pythonanywhere.com"),
    #("Gattus, Luciana", "sin.servidor"),
    ("Grunseid, Viviana Carolina", "vgrunseid.pythonanywhere.com"),
    ("Ibarzabal, Agustina", "agusibar.pythonanywhere.com"),
    ("Jácome Pérez, Fausto Damián", "faustojp.pythonanywhere.com"),
    ("Moretti, Francisco", "fmoretti.pythonanywhere.com"),
    ("Portas, Francisco", "cozaco98.pythonanywhere.com"),
    ("Rey, Augusto Martin", "arey.pythonanywhere.com"),
    ("Sendyk, Uriel Alan", "urizen89.pythonanywhere.com"),
    ("Sosa Suarez, Santiago Andres", "santi23.pythonanywhere.com"),
    ("Tapia, Julian", "jtapia.pythonanywhere.com"),
    ("Truffa, Juan Manuel", "jmtruffa.pythonanywhere.com"),
    ("Vega, Maximiliano", "vegamaximiliano0.pythonanywhere.com"),
    ("Vignolo, Ayrton Cesar", "ayrton88.pythonanywhere.com"),
    ("Vischi Cortés, Paula", "pvischic.pythonanywhere.com"),
]

In [ ]:
for alumno, servidor in servidores:
    print(alumno, end="", flush=True)

    try:
        ###
        endpoint = "/api/version"

        res = requests.get("https://" + servidor + endpoint, timeout=30)
        assert res.status_code == 200, f"debe retornar status code 200 y retorno {res.status_code}"
        assert res.headers.get('content-type') == 'application/json', "no devolvió un objeto tipo json"
        j = res.json()
        assert "version" in j, "el objeto no tiene la clave 'version'"
        assert type(j["version"]) == int, "la version debe ser un número de tipo entero"
        assert j["version"] == 2, f"la version debe ser 2 y es { j['version'] }"

        ###
        endpoint = "/api/init"

        res = requests.get("https://" + servidor + endpoint, timeout=30)
        assert res.status_code == 200, f"debe retornar status_code 200 y retornó {res.status_code}"
        assert res.headers.get('content-type') == 'application/json', "no devolvió un objeto tipo json"
        j = res.json()
        assert "status" in j, "el objeto no tiene la clave 'status'"
        assert j["status"] == "ok", f"el status debe ser 'ok' y devolvió {j['status'] }"

        ###
        endpoint = "/api/recomendaciones"

        con = sqlite3.connect(path.join(THIS_FOLDER, "data_test.db"))
        con.row_factory = sqlite3.Row
        cur = con.cursor()
        lectores = [r["id_lector"] for r in cur.execute("SELECT DISTINCT id_lector FROM interacciones_test").fetchall()][:100]
        con.close()

        res = requests.post("https://" + servidor + endpoint, timeout=60*60, json={"id_lectores": lectores})
        assert res.status_code == 200, f"debe retornar status code 200 y retorno {res.status_code}"
        assert res.headers.get('content-type') == 'application/json', "no devolvió un objeto tipo json"
        j = res.json()
        assert "recomendaciones" in j, "el objeto no tiene la clave 'recomendaciones'"
        assert type(j["recomendaciones"]) == list, f"'recomendaciones' debe ser de tipo lista y es de tipo {type(j['recomendaciones'])}"
        assert len(j["recomendaciones"]) == len(lectores), f"'recomendaciones' debe tener {len(j['recomendaciones'])} elementos y tiene {len(lectores)}"

        con = sqlite3.connect(path.join(THIS_FOLDER, "data.db"))
        con.row_factory = sqlite3.Row
        cur = con.cursor()

        for rec in j["recomendaciones"]:
            assert type(rec) == dict, f"el elemento '{rec}' de las recomendaciones debe ser de tipo dict"

            assert "id_lector" in rec, f"el objeto '{rec}' no tiene la clave 'id_lector'"
            assert "recomendaciones" in rec, f"el objeto '{rec}' no tiene la clave 'recomendaciones'"

            for id_libro in rec["recomendaciones"]:
                res = cur.execute("SELECT * FROM libros WHERE id_libro = ?", [id_libro]).fetchone()
                assert res is not None, f"el id_libro '{id_libro}' de la recomendacion no existe"
        con.close()
    except Exception as e:
        print(f"--> ERROR ({endpoint}):", e)
        continue

    ### metricas
    con = sqlite3.connect(path.join(THIS_FOLDER, "data_test.db"))
    con.row_factory = sqlite3.Row
    cur = con.cursor()

    s = 0.0
    for rec in j["recomendaciones"]:
        gt = [r["id_libro"] for r in cur.execute("SELECT id_libro FROM interacciones_test WHERE id_lector = ?", [rec['id_lector']]).fetchall()]
        s = s + ndcg(gt, rec["recomendaciones"])
    s = s / len(j["recomendaciones"])
    con.close()

    print(" -->", f"{s:2.5f}")